## Moderate Resolution Imaging Spectroradiometer (MODIS) Demo

In this tutorial, we will take a look at some MODIS data. Details of the instrument are outlined here: https://modis.gsfc.nasa.gov/about/

To read in the correct data files, we can use SatPy's `find_files_and_readers` helper function.

In [ ]:
# Ignore warning and debug messages
# NB: If things go wrong, you can turn the debugger on and turn the warnings filter off.
import warnings
warnings.catch_warnings()
warnings.simplefilter("ignore")
from satpy.utils import debug_off; debug_off()


from satpy import find_files_and_readers
from datetime import datetime


path_modis = './data/modis/'
files = find_files_and_readers(start_time=datetime(2023, 4, 17, 22, 50), 
                               end_time=datetime(2023, 4, 17, 22, 55),
                               reader='modis_l1b',
                               base_dir=path_modis)

We can check if the files list is correct by printing it out the dictionary

In [ ]:
print(files)

To get the most out of MODIS data, you need to download 4 files per granule (more details here: https://mcst.gsfc.nasa.gov/sites/default/files/file_attachments/M1054.pdf). The filenames can be interpreted as:

| Terra    | Aqua     | Description |
|----------|----------|-------------|
| MOD02QKM | MYD02QKM | Calibrated Earth View data at 250m resolution.  |
| MOD02HKM | MYD02HKM | Calibrated Earth View data at 500m resolution, including the 250m resolution bands aggregated to 500m resolution. |
| MOD021KM | MYD021KM | Calibrated Earth View data at 1km resolution, including the 250m and 500m resolution bands aggregated to 1km resolution. |
| MOD03    | MYD03    | Geolocation information. |

The filenames contain other information regarding the date and time but it can be difficult to interpret. SatPy allows you to view the metadata in a more friendly format.

The first step when reading in data with SatPy is to load the file list/dictionary into a `Scene` object as follows

In [ ]:
from satpy import Scene

scn = Scene(files)

We can now view which bands are available with various methods that are attached to the `Scene` object.

In [ ]:
scn.available_dataset_names()

From here, we load into memory the satellite datasets of interest. Let's say we want to load band 1. Then we would do:

In [ ]:
scn.load(['1'])  # note that SatPy will complain if we don't load the dataset as a list of strings.

We can now view the metadata pertaining to band 1 like so

In [ ]:
print(scn)

If we want to see the unprojected image, we can simply do:

In [ ]:
scn.show('1')

By default SatPy will display the most sensible calibration (e.g. reflectances for visible channels, brightness temperatures for thermal channels) as a grey scale image. If we want to make a 'pretty picture', we need to create an image composite. To see which composites are available for MODIS we can do:

In [ ]:
scn.available_composite_names()

Let's run the simplest of the true color options which is the `true_color_uncorrected` composite. Now, before we do this we must first resample the Scene object. This is because not all MODIS channels have the same spatial resolution. To resample the data, we can use the `scn.finest_area()` method.

In [ ]:
print(scn.finest_area())

This essentially returns the information that is needed to reproject the coarser resolution bands to the finest resolution bands, which in this case is 250 m. To resample the bands, we do:

In [ ]:
# First, we load the desired composite using our Scene object
scn.load(['true_color_uncorrected'])
# Second, we resmaple the Scene object so that all required bands have the same dimensions (resolution)
lcn = scn.resample(scn.finest_area())  # Here I've used lcn as short-hand for 'local scene'.
# Third, we now can view the resampled composite like so:
lcn.show('true_color_uncorrected')

If we wanted to save the dataset to a geotiff file we can use the writers method `.save_dataset()` method.

In [ ]:
lcn.save_dataset('true_color_uncorrected', filename='./data/geotiff/modis.tif')

Finally, let's say we're interested in a particular region, like Macquarie Island. Often we only know the geographic (latitude/longitude) coordinates. To resample to an area of interest, we can use the pyresample module like so:

In [ ]:
from pyresample import create_area_def

# Let's look at a region that covers Macquarie Isalnd
# extent = [lonmin, lonmax, latmin, latmax]
extent = [158.6, 159.2, -54.8, -54.4]
area_def = create_area_def('Macquarie Island', "+proj=eqc", units="degrees",
                           area_extent=(extent[0], extent[2], extent[1], extent[3]), resolution=0.0001)

This creates an area definition object, which contains information on the projection that we want. Pyresample uses the PROJ conventions for selecting geographic projections (more info here: https://proj.org/). To see what is contained in the `area_def` object we can print it out

In [ ]:
print(area_def)

To resample to our custom area, we simply follow the producure that we used before:

In [ ]:
# Resample to custom area def
scn.load(['true_color_crefl'])  # NB: here I use a true color composite which includes an atmospheric correction.
lcn = scn.resample(area_def)
lcn.show('true_color_crefl')